In [ ]:
# This script performs four main operations:
# 1) Extract cell boundaries
# 2) Keep only large-area boundaries
# 3) Label each boundary with color codes (rc, bc, etc.)
# 4) Generate an image with a transparent background

import cv2
import numpy as np
import os

def get_color_name(bgr):
    """
    Determines the closest reference color (red, green, blue, cyan, magenta, yellow)
    based on Euclidean distance in BGR space.
    """
    reference_colors = {
        "red": (0, 0, 255),
        "green": (0, 255, 0),
        "blue": (255, 0, 0),
        "cyan": (255, 255, 0),
        "magenta": (255, 0, 255),
        "yellow": (0, 255, 255)
    }

    distances = {}
    for name, ref_bgr in reference_colors.items():
        db = bgr[0] - ref_bgr[0]
        dg = bgr[1] - ref_bgr[1]
        dr = bgr[2] - ref_bgr[2]
        distances[name] = db * db + dg * dg + dr * dr

    return min(distances, key=distances.get)

def create_smoothed_color_labeled_outline(
    stereo_img,
    output_directory,
    scale_factor=4.0,
    min_area=100,
    epsilon_factor=0.00001
):
    """
    Upscales the image, smooths each channel, detects edges, and filters small contours.
    Labels each large contour by average color and draws it on a transparent PNG.
    """
    # Upscale the input image
    h, w = stereo_img.shape[:2]
    up_w = int(w * scale_factor)
    up_h = int(h * scale_factor)

    stereo_up = cv2.resize(
        stereo_img,
        (up_w, up_h),
        interpolation=cv2.INTER_LANCZOS4
    )

    # Split into channels
    b, g, r = cv2.split(stereo_up)

    # Smooth each channel
    b_blur = cv2.GaussianBlur(b, (5, 5), 0.7)
    g_blur = cv2.GaussianBlur(g, (5, 5), 0.7)
    r_blur = cv2.GaussianBlur(r, (5, 5), 0.7)

    # Detect edges
    edges_b = cv2.Canny(b_blur, 30, 90)
    edges_g = cv2.Canny(g_blur, 30, 90)
    edges_r = cv2.Canny(r_blur, 30, 90)

    # Combine edges
    edges = cv2.bitwise_or(edges_b, edges_g)
    edges = cv2.bitwise_or(edges, edges_r)

    # Morphological closing (dilate then erode)
    kernel = np.ones((3, 3), np.uint8)
    edges = cv2.dilate(edges, kernel, iterations=1)
    edges = cv2.erode(edges, kernel, iterations=1)

    # Find and filter contours
    contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    filtered_contours = []
    for c in contours:
        area = cv2.contourArea(c)
        if area < min_area:
            continue
        epsilon = epsilon_factor * cv2.arcLength(c, True)
        c_simplified = cv2.approxPolyDP(c, epsilon, True)
        filtered_contours.append(c_simplified)

    # Prepare a black canvas for drawing
    outline_bgr = np.zeros((up_h, up_w, 3), dtype=np.uint8)

    short_names = {
        "red": "rc",
        "green": "gc",
        "blue": "bc",
        "cyan": "cc",
        "magenta": "mc",
        "yellow": "yc"
    }
    color_counts = {}

    # Label each contour
    for contour in filtered_contours:
        # Create a mask for this contour
        mask = np.zeros((up_h, up_w), dtype=np.uint8)
        cv2.drawContours(mask, [contour], -1, 255, -1)

        # Compute average color in this region
        mean_b_val = cv2.mean(b, mask=mask)[0]
        mean_g_val = cv2.mean(g, mask=mask)[0]
        mean_r_val = cv2.mean(r, mask=mask)[0]
        avg_bgr = (mean_b_val, mean_g_val, mean_r_val)

        # Match to the nearest reference color
        color_name = get_color_name(avg_bgr)
        color_counts[color_name] = color_counts.get(color_name, 0) + 1
        cell_num = color_counts[color_name]
        prefix = short_names[color_name]
        cell_label = f"{prefix}{cell_num}"

        # Draw contour outline in white
        cv2.drawContours(outline_bgr, [contour], -1, (255, 255, 255), 1)

        # Label at centroid
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            cv2.putText(
                outline_bgr,
                cell_label,
                (cX, cY),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.3,
                (255, 255, 255),
                1,
                cv2.LINE_AA
            )

    # Convert black background to transparent
    outline_gray = cv2.cvtColor(outline_bgr, cv2.COLOR_BGR2GRAY)
    _, alpha = cv2.threshold(outline_gray, 1, 255, cv2.THRESH_BINARY)
    outline_rgba = cv2.cvtColor(outline_bgr, cv2.COLOR_BGR2BGRA)
    outline_rgba[:, :, 3] = alpha

    # Save the transparent result
    labeled_path = os.path.join(output_directory, "labeled_outline.png")
    cv2.imwrite(labeled_path, outline_rgba)
    print(f"[INFO] Labeled (transparent) outline saved to: {labeled_path}")

def main():
    """
    Provides an example usage of the create_smoothed_color_labeled_outline function.
    Adjust paths and parameters as needed.
    """
    stereo_img_path = "/Users/yonghengwang/Downloads/Cell_Boundary_Cell_Type.png"
    output_directory = "/Users/yonghengwang/Downloads"

    stereo_img = cv2.imread(stereo_img_path)
    if stereo_img is None:
        raise ValueError(f"Could not read image at: {stereo_img_path}")

    create_smoothed_color_labeled_outline(
        stereo_img,
        output_directory,
        scale_factor=4.0,
        min_area=500,
        epsilon_factor=0.00001
    )

if __name__ == "__main__":
    main()


[INFO] Labeled (transparent) outline saved to: /Users/yonghengwang/Downloads/labeled_outline.png
